# pycobra and scikit-learn

This notebook demonstrates pycobras integration with the scikit-learn ecosystem.
We will also give an example of pycobra's performance on some real world data-sets.

In [1]:
from pycobra.cobra import Cobra
from pycobra.ewa import Ewa
from pycobra.diagnostics import Diagnostics
from pycobra.visualisation import Visualisation
import numpy as np
%matplotlib inline

Let's set up a synthetic data-set just to show that the COBRA estimator is scikit-learn compatible.

In [2]:
# setting up our random data-set
rng = np.random.RandomState(1)

# D1 = train machines; D2 = create COBRA; D3 = calibrate epsilon, alpha; D4 = testing
n_features = 20
D1, D2, D3, D4 = 200, 200, 200, 200
D = D1 + D2 + D3 + D4
X = rng.uniform(-1, 1, D * n_features).reshape(D, n_features)
Y = np.power(X[:,1], 2) + np.power(X[:,3], 3) + np.exp(X[:,10]) 
# Y = np.power(X[:,0], 2) + np.power(X[:,1], 3)

# training data-set
X_train = X[:D1 + D2]
X_test = X[D1 + D2 + D3:D1 + D2 + D3 + D4]
X_eps = X[D1 + D2:D1 + D2 + D3]
# for testing
Y_train = Y[:D1 + D2]
Y_test = Y[D1 + D2 + D3:D1 + D2 + D3 + D4]
Y_eps = Y[D1 + D2:D1 + D2 + D3]

Similar to other scikit-learn estimators, we set up our machine by creating an object and then fitting it. 

In [3]:
cobra = Cobra(random_state=rng)

In [4]:
cobra.fit(X_train, Y_train, epsilon=0.5)

Cobra(random_state=<mtrand.RandomState object at 0x1060c7a50>)

We now see if our object can fit into the scikit-learn pipeline and GridSearch - and it can!

In [5]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(Cobra) #passes

### Exponentially Weighted Average Aggregate

Let us also demonstrate the EWA predictor. 

In [6]:
ewa = Ewa()

In [7]:
ewa.fit(X_train, Y_train)

Ewa(random_state=None)

In [8]:
check_estimator(Ewa) #passes

Like the Cobra estimator, Ewa is also a scikit-learn compatible estimator. It also fits into the Visualisation class, like demonstarted in the [notebook](https://github.com/bhargavvader/pycobra/blob/master/notebooks/visualise.ipynb). 

### Predicting?

Like the other scikit-learn predictors, we estimate on data by simply using the `predict()` method.

In [9]:
query = X_test[0].reshape(1, -1)

In [10]:
cobra.predict(query)

array([ 0.6896518])

In [11]:
ewa.predict(query)

array([ 0.67998475])

### Why pycobra?

There are scikit-learn estimators which already perform well in basic regression tasks - why use pycobra?
The Cobra estimator has the advantage of a theoretical bound on it's performance - this means it is supposed to perform at least as well as the estimators used to create it. The Ewa estimator also has similar bounds.

pycobra also lets you compare the scikit-learn estimators used in the aggregation - unlike the ensemble methods for regression which scikit-learn has, pycobra's algorithms is actually built on other scikit-learn like estimators. 

### pycobra for classification

pycobra also implements the classification algorithm as described by Mojirsheibani [1999] Combining Classifiers via Discretization, Journal of the American Statistical Association. 

In [12]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:-20]
y = iris.target[:-20]
X_test = iris.data[-20:]
y_test = iris.target[-20:]

In [13]:
from pycobra.classifiercobra import ClassifierCobra
check_estimator(ClassifierCobra)

In [14]:
cc = ClassifierCobra()

In [15]:
cc.fit(X, y)

ClassifierCobra(random_state=None)

In [23]:
cc.predict(X_test)

array([ 2.,  2.,  2.,  1.,  1.,  2.,  2.,  2.,  1.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.])

### Real-world datasets

We have demonstrated in the regression notebook how pycobra works on synthetic data-sets. Let's see pycobra in action on some scikit-learn regression datasets.

In [ ]:
diabetes = datasets.load_diabetes()

diabetes_X_train = diabetes.data[:-40]
diabetes_X_test = diabetes.data[-20:]
# part of the data to find an appropriate epsilon
diabetes_X_eps = diabetes.data[-40:-20]

diabetes_y_train = diabetes.target[:-40]
diabetes_y_test = diabetes.target[-20:]
diabetes_y_eps = diabetes.target[-40:-20]

We're unaware of what epsilon value to choose for our data-sets so by passing `X_eps` and `y_eps` we can get an idea of what might be a good epsilon value. 

In [ ]:
COBRA_diabetes = Cobra()
COBRA_diabetes.fit(diabetes_X_train, diabetes_y_train, X_epsilon=diabetes_X_eps, y_epsilon=diabetes_y_eps)

Predicting using the COBRA predictor is again similar to using a scikit-learn estimator.

In [ ]:
COBRA_diabetes.predict(diabetes_X_test)

Let's compare our MSEs using the diagnostics class now.

In [ ]:
cobra_diagnostics = Diagnostics(COBRA_diabetes, diabetes_X_test, diabetes_y_test, load_MSE=True)

In [ ]:
cobra_diagnostics.machine_MSE

Let us similarily use COBRA on the Boston housing data set.

In [ ]:
boston = datasets.load_boston()

boston_X_train = boston.data[:-40]
boston_X_test = boston.data[-20:]
boston_X_eps = boston.data[-40:-20]

boston_y_train = boston.target[:-40]
boston_y_test = boston.target[-20:]
boston_y_eps = boston.target[-40:-20]

In [ ]:
COBRA_boston = Cobra()
COBRA_boston.fit(boston_X_train, boston_y_train, X_epsilon=boston_X_eps, y_epsilon=boston_y_eps)

In [ ]:
cobra_diagnostics = Diagnostics(COBRA_boston, boston_X_test, boston_y_test, load_MSE=True)

In [ ]:
cobra_diagnostics.machine_MSE